# Preparing 8p trans effects data for rSEA

## Setup

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
ttest_results_file = "8p_transeffects.tsv"

ttest_results = pd.\
read_csv(ttest_results_file, sep="\t").\
rename(columns={"Name": "protein"}).\
set_index("protein")

In [3]:
ttest_results

,ovarian_Database_ID,brca_Database_ID,luad_Database_ID,lscc_Database_ID,luad_pvalue,hnscc_pvalue,ovarian_pvalue,colon_pvalue,brca_pvalue,lscc_pvalue,luad_diff,hnscc_diff,ovarian_diff,colon_diff,brca_diff,lscc_diff
protein,,,,,,,,,,,,,,,,
A1BG,NP_570602,NP_570602.2,NP_570602.2,NP_570602.2,1.000000,1.0,1.0,1.0,1.0,1.0,-0.442692,-0.037599,-0.294055,-0.119042,-0.102606,0.312395
A2M,NP_000005,NP_000005.2,NP_000005.2|NP_001334353.1|NP_001334354.1|K4JD...,NP_000005.2|NP_001334353.1|NP_001334354.1,1.000000,1.0,1.0,1.0,1.0,1.0,-0.461877,-0.094712,-0.244195,-0.081407,0.331652,0.301145
AAAS,NP_056480,NP_056480.1|NP_001166937.1,NP_056480.1|NP_001166937.1,NP_056480.1|NP_001166937.1,1.000000,1.0,1.0,1.0,1.0,1.0,0.219718,0.020226,0.030730,-0.100606,0.049927,-0.037585
AACS,NP_076417,NP_076417.2|NP_001306769.1|NP_001306768.1,NP_076417.2|NP_001306769.1|NP_001306768.1,NP_076417.2|NP_001306769.1|NP_001306768.1,1.000000,1.0,1.0,1.0,1.0,1.0,-0.103594,0.039000,0.132034,0.089959,0.109607,-0.527445
AAGAB,NP_078942,NP_078942.3|NP_001258815.1,NP_078942.3|NP_001258814.1,NP_078942.3|NP_001258814.1,1.000000,1.0,1.0,1.0,1.0,1.0,-0.119763,-0.026190,0.049414,-0.078345,0.367299,-0.174586
AAMDC,NP_078960,NP_078960.1|NP_001303886.1|NP_001303887.1,NP_001303889.1|NP_001350493.1|NP_001303886.1|N...,NP_001303889.1|NP_001350493.1,1.000000,1.0,1.0,1.0,1.0,1.0,-0.050954,0.052767,0.126284,-0.003264,-0.056287,0.070351
AAMP,NP_001078,NP_001289474.1|NP_001078.2,NP_001289474.1|NP_001078.2,NP_001289474.1|NP_001078.2,1.000000,1.0,1.0,1.0,1.0,1.0,-0.276061,0.048081,0.091424,-0.085926,0.143079,0.328295
AAR2,NP_001258803,NP_001258803.1,NP_001258803.1,NP_001258803.1,1.000000,1.0,1.0,1.0,1.0,1.0,0.062387,-0.014591,0.039881,0.119473,0.167240,-0.148427
AARS2,NP_065796,NP_065796.1,NP_065796.1,NP_065796.1,1.000000,1.0,1.0,1.0,1.0,1.0,0.026319,0.070640,0.040647,0.138317,-0.089839,-0.101160


## Reshape the input dataframe
We want to get our table to have these columns:
- cancer_type
- protein
- Database_ID
- change
- p_value

Since some cancer types have database IDs and some don't, we'll slice out and reshape the info for each cancer type individually.

In [4]:
cancer_types = sorted(ttest_results.columns.to_series().str.split("_", n=1, expand=True)[0].unique())

long_results = pd.DataFrame()

for cancer_type in cancer_types:
    cancer_df = ttest_results.\
    loc[:, ttest_results.columns.str.startswith(cancer_type)].\
    dropna(axis="index", how="all").\
    reset_index(drop=False)
    
    # If the cancer type has database IDs, make a separate column that has them.
    # If not, create a column of NaNs (so that the tables all match)
    if f"{cancer_type}_Database_ID" in cancer_df.columns:
        cancer_df = cancer_df.rename(columns={f"{cancer_type}_Database_ID": "Database_ID"})
    else:
        cancer_df = cancer_df.assign(Database_ID=np.nan)
        
    # Rename the pvalue and diff columns to not have the cancer type
    cancer_df = cancer_df.rename(columns={
        f"{cancer_type}_pvalue": "adj_p",
        f"{cancer_type}_diff": "change"
    }).\
    assign(cancer_type=cancer_type)
    
    # Reorder the columns
    cancer_df = cancer_df[["cancer_type", "protein", "Database_ID", "adj_p", "change"]]
    
    # Append to the overall dataframe
    long_results = long_results.append(cancer_df)

# Drop duplicate rows and reset the index
long_results = long_results[~long_results.duplicated(keep=False)].\
reset_index(drop=True)

In [5]:
long_results

,cancer_type,protein,Database_ID,adj_p,change
0,brca,A1BG,NP_570602.2,1.0,-0.102606
1,brca,A2M,NP_000005.2,1.0,0.331652
2,brca,AAAS,NP_056480.1|NP_001166937.1,1.0,0.049927
3,brca,AACS,NP_076417.2|NP_001306769.1|NP_001306768.1,1.0,0.109607
4,brca,AAGAB,NP_078942.3|NP_001258815.1,1.0,0.367299
5,brca,AAMDC,NP_078960.1|NP_001303886.1|NP_001303887.1,1.0,-0.056287
6,brca,AAMP,NP_001289474.1|NP_001078.2,1.0,0.143079
7,brca,AAR2,NP_001258803.1,1.0,0.167240
8,brca,AARS2,NP_065796.1,1.0,-0.089839
9,brca,AARSD1,NP_001248363.1|NP_001129514.2|NP_079543.1,1.0,-0.221135


## Select the proteins with a significant change

In [6]:
prots = long_results[long_results["adj_p"] <= 0.05].reset_index(drop=True)

## Find how many cancers each protein was different in

In [7]:
prots_summary = prots.groupby("protein").agg(**{
    "cancers": ("cancer_type", lambda x: x.sort_values().drop_duplicates(keep="first").tolist())
})

prots_summary = prots_summary.\
assign(
    num_cancers=prots_summary["cancers"].apply(len),
    tmp_sort=prots_summary["cancers"].apply(lambda x: "".join(x))
).\
sort_values(by=["num_cancers", "tmp_sort"], ascending=[False, True]).\
drop(columns="tmp_sort")

prots_summary

,cancers,num_cancers
protein,,
CNOT8,"[brca, luad]",2
ATP6V1A,[colon],1
ATP6V1E1,[colon],1
ATP6V1G1,[colon],1
ATP6V1H,[colon],1
CRELD2,[colon],1
NUCB2,[colon],1
GPNMB,[hnscc],1
EGFR,[luad],1
